<h1>Спринт "Кофейни в Москве"</h1>
<br>
## Проектная работа Курзенева Александра

# <h3>Описание проекта</h3>



Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.

Для начала они просят вас — аналитика — подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

Постарайтесь сделать презентацию информативной и лаконичной. Её структура и оформление сильно влияют на восприятие информации читателями вашего исследования. Выбирать инструменты (matplotlib, seaborn и другие) и типы визуализаций вы можете самостоятельно.

Вам доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках. Она носит исключительно справочный характер.

## Описание данных

Файл moscow_places.csv:
- name — название заведения;
- address — адрес заведения;
- category — категория заведения, например «кафе», «пиццерия» или «кофейня»;
- hours — информация о днях и часах работы;
- lat — широта географической точки, в которой находится заведение;
- lng — долгота географической точки, в которой находится заведение;
- rating — рейтинг заведения по оценкам пользователей в Яндекс Картах (высшая оценка — 5.0);
price — категория цен в заведении, например «средние», «ниже среднего», «выше среднего» и так далее;
- avg_bill — строка, которая хранит среднюю стоимость заказа в виде диапазона, например:
«Средний счёт: 1000–1500 ₽»;
«Цена чашки капучино: 130–220 ₽»;
«Цена бокала пива: 400–600 ₽».
и так далее;
- middle_avg_bill — число с оценкой среднего чека, которое указано только для значений из столбца  avg_bill, начинающихся с подстроки «Средний счёт»:
- middle_coffee_cup — число с оценкой одной чашки капучино, которое указано только для значений из столбца avg_bill, начинающихся с подстроки «Цена одной чашки капучино»:
- chain — число, выраженное 0 или 1, которое показывает, является ли заведение сетевым (для маленьких сетей могут встречаться ошибки);
- district — административный район, в котором находится заведение, например Центральный административный округ;
- seats — количество посадочных мест.
Инструкция по выполнению проек

# Загрузим данные и изучите общую информацию

In [1]:
#!pip3 install folium
#!pip3 install plotly-express

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import plotly.express as px
import math as mth
import re
import scipy.stats as stats
import warnings

#from google.colab import drive 
from matplotlib import pyplot as plt
from scipy import stats as st
from folium import Map, Choropleth, Marker
from folium.plugins import MarkerCluster
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

In [3]:
try:
    print('Загружаем локально...')
    #drive.mount('/content/gdrive')
    #data = pd.read_csv('gdrive/My Drive/Colab Notebooks/datasets/moscow_places.csv')
    data = pd.read_csv('/datasets/moscow_places.csv')
except:
    print('Локально не нашли, загружаем с сервера...')
    data = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')
print('Done')

Загружаем локально...
Локально не нашли, загружаем с сервера...
Done


In [4]:
data.head()

,name,category,address,district,hours,lat,lng,rating,price,avg_bill,middle_avg_bill,middle_coffee_cup,chain,seats
0,WoWфли,кафе,"Москва, улица Дыбенко, 7/1",Северный административный округ,"ежедневно, 10:00–22:00",55.878494,37.478860,5.0,NaN,NaN,NaN,NaN,0,NaN
1,Четыре комнаты,ресторан,"Москва, улица Дыбенко, 36, корп. 1",Северный административный округ,"ежедневно, 10:00–22:00",55.875801,37.484479,4.5,выше среднего,Средний счёт:1500–1600 ₽,1550.0,NaN,0,4.0
2,Хазри,кафе,"Москва, Клязьминская улица, 15",Северный административный округ,"пн-чт 11:00–02:00; пт,сб 11:00–05:00; вс 11:00...",55.889146,37.525901,4.6,средние,Средний счёт:от 1000 ₽,1000.0,NaN,0,45.0
3,Dormouse Coffee Shop,кофейня,"Москва, улица Маршала Федоренко, 12",Северный административный округ,"ежедневно, 09:00–22:00",55.881608,37.488860,5.0,NaN,Цена чашки капучино:155–185 ₽,NaN,170.0,0,NaN
4,Иль Марко,пиццерия,"Москва, Правобережная улица, 1Б",Северный административный округ,"ежедневно, 10:00–22:00",55.881166,37.449357,5.0,средние,Средний счёт:400–600 ₽,500.0,NaN,1,148.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8406 entries, 0 to 8405
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               8406 non-null   object 
 1   category           8406 non-null   object 
 2   address            8406 non-null   object 
 3   district           8406 non-null   object 
 4   hours              7870 non-null   object 
 5   lat                8406 non-null   float64
 6   lng                8406 non-null   float64
 7   rating             8406 non-null   float64
 8   price              3315 non-null   object 
 9   avg_bill           3816 non-null   object 
 10  middle_avg_bill    3149 non-null   float64
 11  middle_coffee_cup  535 non-null    float64
 12  chain              8406 non-null   int64  
 13  seats              4795 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 919.5+ KB


Сколько уникальных хаведений в датасете:

In [6]:
data['name'].nunique()

5614

Сколько пропусков вданных

In [7]:
data.isna().sum()

name                    0
category                0
address                 0
district                0
hours                 536
lat                     0
lng                     0
rating                  0
price                5091
avg_bill             4590
middle_avg_bill      5257
middle_coffee_cup    7871
chain                   0
seats                3611
dtype: int64

## Общий вывод

Некоторые столбцы имеют много пропусков. Заполнить их не представляеться возможным, но надо это учитывать при последующем анализе.

# Выполним предобработку данных

## Обработка дубликатов и пропусков

Изучите, есть ли дубликаты в данных. Поищите пропуски: встречаются ли они, в каких столбцах? Можно ли их обработать или оставить как есть?

In [8]:
print('Количество полных дублей',data.duplicated().sum())
print('Количество строк до',len(data))
data.drop_duplicates(['name', 'address'])
print('Количество строк после',len(data))



Количество полных дублей 0
Количество строк до 8406
Количество строк после 8406


## Создадим столбцы street и is_24/7

Создадим столбец street с названиями улиц из столбца с адресом.

Создадим столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
(True — если заведение работает ежедневно и круглосуточно и False — в противоположном случае.

In [9]:
data['street'] = data['address'].apply(lambda x: x.split(', ')[1])
data['is_24_7'] = data['hours'].isin(['ежедневно, круглосуточно'])

## Общий вывод

В датасете нет дублей, это хорошо. Но есть много пропусков, частично это обусловлено не совсем правильной струкутрой данных(столбцы *middle_avg_bill* и *middle_coffee_cup* как бы дополняют столбец *avg_bill* ), так и просто отсутствующими данными. Видимо не все заведения их предоставили. Заполнить отсутствующие данные не представляеться возможным, поэтому работать будем с тем, что есть.

# Анализ данных

## Какие категории заведений представлены в данных?
Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. 



In [10]:
data_cat = data.pivot_table(index=['category'], values=['name','seats'], aggfunc=['count','sum']).reset_index()
data_cat.columns =['category','names', 'seats', 'seats_sum']
fig = px.histogram(data_cat, x='category', y='names',text_auto=True,height=600,color_discrete_sequence=[px.colors.qualitative.Pastel[0]],)
fig.update_layout(title='Количество обьектов по категориям',
                   xaxis_title='Категории обьектов',
                   yaxis_title='Количество',
                   xaxis={'categoryorder':'total ascending'})
fig.show();

**Вывод** Больше всего кафе и ресторанов, меньше всего столовых и булочных. 

## Исследуем количество посадочных мест по категориям 



In [11]:
print('Всего посадочных мест в Москве',data['seats'].sum())

Всего посадочных мест в Москве 519882.0


In [12]:
fig = px.histogram(data_cat, x='category', y='seats_sum',text_auto='.3s',
             height=600,color_discrete_sequence=[px.colors.qualitative.Pastel[1]],)
fig.update_layout(title='Количество посадочных мест по категориям',
                   xaxis_title='Категории обьектов',
                   yaxis_title='Количество мест',
                   xaxis={'categoryorder':'total ascending'})
fig.show();

In [13]:
fig = px.box(data, x='category', y='seats', color='category',  height=600,)
fig.update_layout(title='Количество посадочных мест по категориям',
                   xaxis_title='Категории обьектов',
                   yaxis_title='Количество мест',
                   )
fig.show();

In [14]:
data.query('seats>900')

,name,category,address,district,hours,lat,lng,rating,price,avg_bill,middle_avg_bill,middle_coffee_cup,chain,seats,street,is_24_7
2713,Ваня и Гоги,"бар,паб","Москва, Измайловское шоссе, 71, корп. А",Восточный административный округ,"ежедневно, 11:00–06:00",55.789842,37.750282,4.2,высокие,Средний счёт:1000–2500 ₽,1750.0,NaN,0,1040.0,Измайловское шоссе,False
2722,Маргарита,быстрое питание,"Москва, Измайловское шоссе, 71, корп. А",Восточный административный округ,"ежедневно, 10:00–22:00",55.789527,37.752004,4.3,NaN,NaN,NaN,NaN,1,1040.0,Измайловское шоссе,False
2770,Шоколадница,кофейня,"Москва, Измайловское шоссе, 71, корп. А",Восточный административный округ,"пн-ср 11:00–23:00; чт 11:00–00:00; пт,сб 11:00...",55.789903,37.749822,4.1,NaN,NaN,NaN,NaN,1,1040.0,Измайловское шоссе,False
2966,Матрешка,кафе,"Москва, Измайловское шоссе, 71, корп. А",Восточный административный округ,NaN,55.789867,37.749656,4.0,NaN,NaN,NaN,NaN,0,1040.0,Измайловское шоссе,False
4180,Eataly,"бар,паб","Москва, Киевская улица, 2",Западный административный округ,"ежедневно, 12:00–23:00",55.743405,37.562535,4.6,NaN,NaN,NaN,NaN,0,920.0,Киевская улица,False
4231,РестоБар Argomento,столовая,"Москва, Кутузовский проспект, 41, стр. 1",Западный административный округ,"ежедневно, 12:00–23:00",55.738237,37.531819,4.2,высокие,Средний счёт:2500–5000 ₽,3750.0,NaN,0,1200.0,Кутузовский проспект,False
4245,Стейк & Бургер,кафе,"Москва, Киевская улица, 2",Западный административный округ,"ежедневно, 09:00–21:00",55.742953,37.561872,4.2,NaN,NaN,NaN,NaN,1,920.0,Киевская улица,False
6518,DelonixCafe,ресторан,"Москва, проспект Вернадского, 94, корп. 1",Западный административный округ,"ежедневно, круглосуточно",55.652577,37.475730,4.1,высокие,Средний счёт:1500–2000 ₽,1750.0,NaN,0,1288.0,проспект Вернадского,True
6524,Ян Примус,ресторан,"Москва, проспект Вернадского, 121, корп. 1",Западный административный округ,"пн-чт 12:00–00:00; пт,сб 12:00–02:00; вс 12:00...",55.657166,37.481519,4.5,выше среднего,Средний счёт:1500 ₽,1500.0,NaN,1,1288.0,проспект Вернадского,False
6574,Мюнгер,пиццерия,"Москва, проспект Вернадского, 97, корп. 1",Западный административный округ,"пн-пт 08:00–21:00; сб,вс 10:00–21:00",55.667505,37.491001,4.8,NaN,NaN,NaN,NaN,1,1288.0,проспект Вернадского,False


**Вывод**. Больше всего посадочных мест в ресторанах и кафе, меньше всего мест в столовых и булочных

## Рассмотрим соотношение сетевых и несетевых заведений

In [15]:
data_chain = data.pivot_table(index=['name'], values=['chain'], aggfunc=['count', 'sum'])
data_chain.columns =['chain_count', 'chain_sum']
print('Кол-во заведения в одном экземпляре, но написаны что сетевые',len(data_chain.query('chain_count==1 & chain_sum!=0')))
print('Кол-во заведения в многих экземплярах , но написаны что не сетевые',len(data_chain.query('chain_count>1 & chain_sum==0')))

Кол-во заведения в одном экземпляре, но написаны что сетевые 64
Кол-во заведения в многих экземплярах , но написаны что не сетевые 16


Сетевые заведения в одном экземпляре, скорее всего остальные находяться не в Москве.

Насчет 16 сетевых, но в нескольких экземплярах, возможно у них просто одинаковые названия.

Рассмотрим соотношение сетевых и несетевых заведений в датасете. 

In [16]:
#plt.style.use('fivethirtyeight') 
#plt.figure(figsize=(12, 8))
#plt.pie(data.groupby('chain')['chain'].agg('count'), labels=['Не сетевые','Сетевые'], shadow=True, autopct='%1.1f%%')
#fig.title('Соотношение сетевых заведений и не сетевых2') 
#plt.show();

fig = px.pie(data.groupby('chain')['chain'].agg('count'), values='chain', width=600,
             names=['Не сетевые','Сетевые'], title='Соотношение сетевых заведений и не сетевых')
fig.show();

**Вывод**. Не сетевых заведений больше.

## Какие категории заведений чаще являются сетевыми? 

In [17]:
data_cat_chain = data.pivot_table(index=['category','chain'], values=['name'], aggfunc='count').reset_index()
fig = px.histogram(data_cat_chain, x='category', y='name',color='chain', text_auto='.2s', height=600,)
fig.update_layout(title='Количество обьектов по категориям',
                   xaxis_title='Категории обьектов',
                   legend_title="Сетевые",
                   yaxis_title='Количество',
                  xaxis={'categoryorder':'total ascending'})
fig.show();


#plt.style.use('ggplot') 
#plt.figure(figsize=(12, 8))
#sns.barplot(x='category', y='name', hue='chain', data=data_cat_chain)
#plt.title('Количество обьектов по категориям') 
#plt.xlabel('Категории обьектов') 
#plt.ylabel('Количество')
#plt.legend(['Не сетевые', 'Сетевые']);
#plt.show();

**Вывод**. Сетевых больше чем не сетевых кофейн и булочных, кафе гораздо бельше не сетевых. Как будет показано ниже кафе часто так и называют "Кафе"

## Топ-15 популярных сетей в Москве

Сгруппируем данные по названиям заведений и найдем топ-15 популярных сетей в Москве.  

In [18]:
px.colors.qualitative.Pastel[0]

'rgb(102, 197, 204)'

In [19]:
data_name = data.groupby('name')['name'].agg('count').sort_values(ascending=False).head(15)
fig = px.histogram(data_name, # загружаем данные и заново их сортируем
             x=data_name, # указываем столбец с данными для оси X
             y=data_name.index, # указываем столбец с данными для оси Y
             #color=data_name.index,
             color_discrete_sequence=[px.colors.qualitative.Pastel[2]], text_auto=True,)
fig.update_layout(title='ТОП-15 заведений в Москве',
                   xaxis_title='Количество заведений',
                   yaxis_title='Название заведения',
                   yaxis={'categoryorder':'total ascending'})
fig.show() # выводим график

Очень много сетевых кафе и кофеен.

Отобразим  общее количество заведений и количество заведений каждой категории по районам.

In [20]:
data_okr = data.pivot_table(index=['district','category'], values=['name'], aggfunc='count').reset_index()
fig = px.histogram(data_okr, x='category', y='name',color='district',
              barmode='group', height=600)
fig.update_layout(title='Распределение категорий заведений по округам',
                   yaxis_title='Количество заведений',
                   xaxis_title='Категории / округа')
fig.show()

**Вывод.** Чатсто владельцы не паряться и так и называют заведение "Кафе". 

## Общее количество заведений и количество заведений каждой категории по районам

Какие административные районы Москвы присутствуют в датасете? Отобразим общее количество заведений и количество заведений каждой категории по районам.

In [21]:
fig = px.histogram(data_okr, x='district', y='name',color='category',
              barmode='group', height=900)
fig.update_layout(title='Распределение заведений по районам',
                   yaxis_title ='Количество заведений',
                   xaxis_title='Районы / категории')
fig.show()

**Вывод.** Как и следовало ожидать, больше всего заведений в самом богатом Центральном районе Москвы. Самые популярные там это рестораны,кафе и кофейни. Остальные районы выглядят примерно одинаково.

##  Распределение средних рейтингов по категориям заведений

Визуализируем распределение средних рейтингов по категориям заведений. 

In [22]:
data_cat_raiting = data.pivot_table(index=['category'], values=['rating'], aggfunc=['mean']).reset_index()
data_cat_raiting.columns =['category','raiting_m']
fig = px.histogram(data_cat_raiting, x='category', y='raiting_m',color='category', text_auto='.3s',
            height=600, log_y=True,color_discrete_sequence=[px.colors.qualitative.Pastel[3]])
fig.update_layout(title='Средний рейтинг по категориям',
                   xaxis_title='Категории обьектов',
                   yaxis_title='Средний рейтинг',
                   xaxis={'categoryorder':'total ascending'},)
fig.show();

**Вывод.** Если стоить график в логарифмическом масштабе, то различия видны, но на самом деле они минимальны. Все виды заведений имеют хорошие рейтинги. Самый высокий из них у баров и пабов, а самый низкий у заведений быстрого питания, что похоже на правду.

## Построим картограмму со средним рейтингом заведений каждого района

Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. 

In [23]:
data_district_rating = data.pivot_table(index=['district'], values=['rating'], aggfunc=['mean']).reset_index()
data_district_rating.columns =['district','rating']
# загружаем JSON-файл с границами округов Москвы
state_geo = 'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_district_rating,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='Purples',
    fill_opacity=0.8,
    legend_name='Средний рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

**Вывод. ** Самый высокий средний рейтинг в центре Москвы.

## Отобразим все заведения датасета на карте с помощью кластеров средствами библиотеки folium.

In [24]:
# moscow_lat - широта, moscow_lng – долгота
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)    


data.apply(create_clusters, axis=1)

m

Output hidden; open in https://colab.research.google.com to view.

## Найдем топ-15 улиц по количеству заведений. 

Построbv график распределения количества заведений и их категорий по этим улицам. 

In [25]:
data_street15 = data.groupby('street')['street'].agg('count').sort_values(ascending=False).head(15)
fig = px.histogram(data_street15, 
             x=data_street15, 
             y=data_street15.index, 
             color_discrete_sequence=[px.colors.qualitative.Pastel[4]],
             text_auto=True,)
fig.update_layout(title='ТОП-15 улиц по количеству заведений в Москве',
                   xaxis_title='Количество заведений на улице',
                   yaxis_title='Название заведения',
                   yaxis={'categoryorder':'total ascending'})
fig.show() 

На Проспекте Мира так много заведений, так как это длинная улица и к тому же заведения находящиеся на ВДНХ тоже имеют адрес Проспект Мира

Отобразим на одном графике категории заведений по улицам из топ 15

In [26]:
data_15 = data[data['street'].isin(data_street15.index)]
data_okr15 = data_15.pivot_table(index=['street','category'], values=['name'], aggfunc='count').reset_index()
fig = px.histogram(data_okr15, x='street', y='name',color='category',
              barmode='group', height=900)
fig.update_layout(title='Распределение заведений по улицам из топ-15',
                   yaxis_title ='Количество заведений',
                   xaxis_title='Улицы / категории')
fig.show()

Кто бы мог подумать, что на МКАД есть рестораны, но нет пиццерий.

## Найдем улицы, на которых находится только один объект общепита. 

Что можно сказать об этих заведениях?

Посмотрим на описания:

In [27]:
data_stereet = data.pivot_table(index=['street'], values=['name'], aggfunc=['count']).reset_index()
data_stereet.columns =['street','name_c']
data_stereet1 = data[data['street'].isin(data_stereet.query('name_c==1')['street'])]



print('Все заведения')
print('Всего их',len(data))
print(data[['rating','seats','middle_avg_bill','middle_coffee_cup']].describe())
print()
print('Единственные на улице заведения')
print('Всего их',len(data_stereet1))
print(data_stereet1[['rating','seats','middle_avg_bill','middle_coffee_cup']].describe())

m = Map(location=[moscow_lat, moscow_lng], zoom_start=10,)
marker_cluster = MarkerCluster().add_to(m)

data_stereet1.apply(create_clusters, axis=1)

m

Все заведения
Всего их 8406
            rating        seats  middle_avg_bill  middle_coffee_cup
count  8406.000000  4795.000000      3149.000000         535.000000
mean      4.229895   108.421689       958.053668         174.721495
std       0.470348   122.833396      1009.732845          88.951103
min       1.000000     0.000000         0.000000          60.000000
25%       4.100000    40.000000       375.000000         124.500000
50%       4.300000    75.000000       750.000000         169.000000
75%       4.400000   140.000000      1250.000000         225.000000
max       5.000000  1288.000000     35000.000000        1568.000000

Единственные на улице заведения
Всего их 458
           rating       seats  middle_avg_bill  middle_coffee_cup
count  458.000000  156.000000       165.000000          24.000000
mean     4.236681   59.115385       959.830303         186.166667
std      0.461952   48.274880       999.518083          55.544472
min      1.000000    0.000000        67.000000    

**Вывод.**

Как видно на карте, такие заведения достаточно равномерно расположены по всей Москве.

- Средний рейтинг таких заведений такой же как и в общем по Москве
- Среднее количество мест почти в два раза меньше чем в общем по Москве
- Средний чек немного поменьше чем средний
- Чашка кофе стоит немного дороже


## Построим фоновую картограмму средних чеков для каждого района

Значения средних чеков заведений хранятся в столбце middle_avg_bill. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитайте медиану этого столбца для каждого района. Используйте это значение в качестве ценового индикатора района. Постройте фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируйте цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?

In [28]:
data_avg_bill = data.pivot_table(index=['district'], values=['middle_avg_bill'], aggfunc=['mean']).reset_index()
data_avg_bill.columns =['district','middle_avg_bill']

m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=data_avg_bill,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='Purples',
    fill_opacity=0.8,
    legend_name='Средний чек заведений по районам',
).add_to(m)

# выводим карту
m

**Вывод.**  Сразу видно, где обитают богатые москвичи. Это центр и запад Москвы.

## Реитинги 

Исследуем особенности заведений с плохими рейтингами, средние чеки в таких местах и распределение по категориям заведений.

In [29]:
fig = px.histogram(data, x='rating', height=600,color_discrete_sequence=[px.colors.qualitative.Pastel[5]])
fig.update_layout(title='Распределение рейтингов заведений',
                   yaxis_title ='Количество заведений',
                   xaxis_title='Рейтинг')
fig.show()

In [30]:
percent = np.percentile(data['rating'], 5)
print('Выведем на карту заведения с 5% самым плохим рейтингом',percent)
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10,)
data.query('rating<=@percent').apply(create_marker, axis=1)
m

Выведем на карту заведения с 5% самым плохим рейтингом 3.4


NameError: ignored

In [ ]:
print('Все заведения')
print('Всего их',len(data))
print(data[['rating','seats','middle_avg_bill','middle_coffee_cup']].describe())
print()
print('Заведения с рейтингом меньше',percent)
print('Всего их',len(data.query('rating<=@percent')))
print(data.query('rating<=@percent')[['rating','seats','middle_avg_bill','middle_coffee_cup']].describe())

**Вывод.** В центре заведений с плохим рейтингом практически нет. В условиях высокой арендной платы, выживают сильнейшие.

- Среднее количество мест на треть меньше, чем в общем по Москве
- Средний чек на треть меньше
- Чашка кофе стоит на треть дешевле

Очевидно, что поддерживать высокий рейтинг заведения приносит больше прибыли

## Общий вывод

Мы проанализировали заведения общественного питания Москвы.
Удалось выяснить, что больше всего кафе и ресторанов, меньше всего столовых и булочных.
Всего посадочных мест в Москве 519882. Больше всего посадочных мест в ресторанах и кафе, меньше всего мест в столовых и булочных. Сетевых заведений меньше, чем не сетевых.

Больше всего заведений в самом богатом Центральном районе Москвы. Самые популярные там это рестораны,кафе и кофейни. Остальные районы выглядят примерно одинаково. Также самый высокий средний рейтинг в центре Москвы. Также в центре самый высокий средний чек.

На Проспекте Мира так много заведений, так как это длинная улица и к тому же заведения находящиеся на ВДНХ тоже имеют адрес Проспект Мира

Заведения, единственные на улице достаточно равномерно расположены по всей Москве.
- Средний рейтинг таких заведений такой же как и в общем по Москве
- Среднее количество мест почти в два раза меньше чем в общем по Москве
- Средний чек немного поменьше чем средний
- Чашка кофе стоит немного дороже


В центре заведений с плохим рейтингом практически нет. В условиях высокой арендной платы, выживают сильнейшие.

- Среднее количество мест на треть меньше, чем в общем по Москве
- Средний чек на треть меньше
- Чашка кофе стоит на треть дешевле



# Детализируем исследование: открытие кофейни


Основателям фонда «Shut Up and Take My Money» не даёт покоя успех сериала «Друзья». Их мечта — открыть такую же крутую и доступную, как «Central Perk», кофейню в Москве. Будем считать, что заказчики не боятся конкуренции в этой сфере, ведь кофеен в больших городах уже достаточно. Попробуйте определить, осуществима ли мечта клиентов.

## Сколько всего кофеен в датасете? 

В каких районах их больше всего, каковы особенности их расположения?

In [ ]:
data_coffe = data.query('category=="кофейня"')
print('Кофейни')
print('Всего их',len(data_coffe))
print(data_coffe[['rating','seats','middle_avg_bill','middle_coffee_cup']].describe())

data_coffe_d = data_coffe.pivot_table(index=['district'], values=['name','rating','middle_coffee_cup'], aggfunc=['count','mean']).reset_index()
data_coffe_d.columns =['district','mcc_c','names_c','rating_c','mcc_m','rating_m']

fig = px.histogram(data_coffe_d, y='district', x='names_c',color_discrete_sequence=[px.colors.qualitative.Pastel[6]], text_auto=True,
            height=600)
fig.update_layout(title='Количество кафе по районам',
                   yaxis_title='Район Москвы',
                   xaxis_title='Количество',
                   yaxis={'categoryorder':'total ascending'})
fig.show();

**Вывод.** Кофеен больше всего в Центральном районе Москвы, меньше всего в Северо-Западном округе

## Есть ли круглосуточные кофейни?

In [ ]:
fig = px.pie(data_coffe.groupby('is_24_7')['is_24_7'].agg('count'), values='is_24_7', width=600,
             names=['Не круглосуточные','Круглосуточные'], title='Режим работы кофеен')
fig.show();

In [ ]:
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10,)
data_coffe.query('is_24_7').apply(create_marker, axis=1)
m

**Вывод.** Круглосуточных кофеен не так уж и много, всего 4% от их числа, и почти все они находяться в Центральном округе.

## Какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
fig = px.histogram(data_coffe_d, y='district', x='rating_m', text_auto='.3s',log_x=True, 
                   color_discrete_sequence=[px.colors.qualitative.Pastel[7]],height=600)
fig.update_layout(title='Средние рейтинги кафе по районам',
                   yaxis_title='Район Москвы',
                   xaxis_title='Рейтинг',
                   yaxis={'categoryorder':'total ascending'})
fig.show();

**Вывод.** Средний рейтинг у кофеен ниибольший в Центральном округе, наименьший  в Западном.

## На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
fig = px.histogram(data_coffe_d, y='district', x='mcc_m',
                   color_discrete_sequence=[px.colors.qualitative.Pastel[8]],text_auto='.5s',log_x=True, height=600,)
fig.update_layout(title='Средняя стоимость чашки кофе по районам',
                   yaxis_title='Район Москвы',
                   xaxis_title='Медианная стоимость чашки кофе ',
                   yaxis={'categoryorder':'total ascending'})
fig.show();

**Вывод.** В западном округе самая дорогая чашка, а самая дешевая  в Юго-Восточном

## Общий вывод

Средний рейтинг у кофеен ниибольший в Центральном округе, наименьший в Западном, в то же время, стоимость чашки кофе в Западном округе самая высокая, число кофеен находиться на среднем уровне. Наверно хорошей идеей будет открыть круглосуточную кофейню в этом округе.

# Презентация и итог


**Итог**

Мы проанализировали заведения общественного питания Москвы.
Удалось выяснить, что больше всего кафе и ресторанов, меньше всего столовых и булочных.
Всего посадочных мест в Москве 519882. Больше всего посадочных мест в ресторанах и кафе, меньше всего мест в столовых и булочных. Сетевых заведений меньше, чем не сетевых.

Больше всего заведений в самом богатом Центральном районе Москвы. Самые популярные там это рестораны,кафе и кофейни. Остальные районы выглядят примерно одинаково. Также самый высокий средний рейтинг в центре Москвы. Также в центре самый высокий средний чек.

На Проспекте Мира так много заведений, так как это длинная улица и к тому же заведения находящиеся на ВДНХ тоже имеют адрес Проспект Мира

Заведения, единственные на улице достаточно равномерно расположены по всей Москве.
- Средний рейтинг таких заведений такой же как и в общем по Москве
- Среднее количество мест почти в два раза меньше чем в общем по Москве
- Средний чек немного поменьше чем средний
- Чашка кофе стоит немного дороже


В центре заведений с плохим рейтингом практически нет. В условиях высокой арендной платы, выживают сильнейшие.

- Среднее количество мест на треть меньше, чем в общем по Москве
- Средний чек на треть меньше
- Чашка кофе стоит на треть дешевле



Если же выбирать из кофеен, то средний рейтинг у кофеен ниибольший в Центральном округе, наименьший в Западном, в то же время, стоимость чашки кофе в Западном округе самая высокая, число кофеен находиться на среднем уровне. 

Хорошей идеей будет открыть круглосуточную кофейню в этом округе.